In [193]:
import datetime
import hyperopt
import itertools
import math
import numpy
import operator
import pandas
import random
import requests
from IPython.display import Audio
from sklearn import (
    calibration,
    ensemble,
    linear_model,
    metrics,
    model_selection,
    naive_bayes,
    neighbors,
    neural_network,
    svm,
)
import time
import tqdm
import xgboost

### Read and Filter

In [207]:
dtype = {'smoke': float, 'alco': float, 'active': float}

def fix_ap(value):
    value = abs(value)
    while value > 500.0:
        value /= 10.0
    return value

def read_csv(filename):
    frame = pandas.read_csv(filename, sep=';', header=0, na_values='None', dtype=dtype).drop(['id'], axis=1)
    
    frame = pandas.get_dummies(frame, columns=[
        # 'smoke',
        # 'active',
        # 'alco',
        # 'gender',
        # 'cholesterol',
        # 'gluc',
    ])
    frame = frame.assign(
        bmi=(frame['weight'] / frame['height'] ** 2),
        # aged_smoke_0=(frame['smoke_0.0'] * frame['age']),
        # aged_smoke_1=(frame['smoke_1.0'] * frame['age']),
        # aged_active_0=(frame['active_0.0'] * frame['age']),
        # aged_active_1=(frame['active_1.0'] * frame['age']),
        # aged_alco_0=(frame['alco_0.0'] * frame['age']),
        # aged_alco_1=(frame['alco_1.0'] * frame['age']),
        # smoke_1_active_0=(frame['smoke_1.0'] * frame['active_0.0']),
    )
    frame = frame.drop([
        # 'smoke',
        # 'alco',
        # 'active',
    ], axis=1)
    
    frame['ap_hi'] = frame['ap_hi'].apply(fix_ap)
    
    return frame

In [208]:
train = read_csv('train.csv')

X = train.drop('cardio', axis=1).values
y = train['cardio'].values
print(f'X: {X.shape}')
print(f'y: {y.shape}')

X: (70000, 12)
y: (70000,)


### Hyper-parameter Optimisation

In [372]:
def cv():
    return model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

def print_importances(estimator):
    for name, importance in sorted(zip(test, estimator.feature_importances_), key=operator.itemgetter(1), reverse=True):
        print(f'{name}: {importance:.7f}')

def make_int_grid(params, param_name, delta):
    n = params.get(param_name, 1)
    return {param_name: list(range(max(1, n - delta), n + delta + 1))}

def make_float_grid(params, param_name, delta, min_, max_, default=0.0):
    x = params.get(param_name, default)
    return {param_name: sorted({max(min_, x - delta), x, min(max_, x + delta)})}

def search(estimator, X, y, params, **param_grid):
    start_time = time.time()
    grid_search = model_selection.GridSearchCV(
        estimator.set_params(**params),
        param_grid,
        scoring='neg_log_loss',
        refit=False,
        cv=cv(),
    ).fit(X, y)
    params.update(grid_search.best_params_)
    print(f'[{(time.time() - start_time):.0f}s] {grid_search.best_score_:.10f} {grid_search.best_params_} | {param_grid}')

def search_xgb(X, y, params):
    estimator = xgboost.XGBClassifier(nthread=4, seed=0)
    for _ in range(100):
        print()
        searches = [
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'colsample_bytree', 0.01, 0.5, 1.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'subsample', 0.01, 0.5, 1.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'base_score', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'scale_pos_weight', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'reg_lambda', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'reg_alpha', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'gamma', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_int_grid(params, 'max_depth', 1)),
            lambda: search(estimator, X, y, params, **make_int_grid(params, 'n_estimators', 3)),
        ]
        random.shuffle(searches)
        last_params = dict(params)
        for _search in searches:
            _search()
        if params == last_params:
            break
    
    print('---------------------------------------------------')
    print(params)
    return estimator.set_params(**params).fit(X, y)

### Predict `smoke`, `alco` and `active`

In [296]:
X_helper = train.drop(['smoke', 'alco', 'active', 'cardio'], axis=1).values

y_smoke = train['smoke'].values
y_alco = train['alco'].values
y_active = train['active'].values

In [360]:
# smoke_params = {'base_score': 0.53, 'scale_pos_weight': 1.0, 'reg_lambda': 0.98, 'reg_alpha': 0.01, 'gamma': 0.03999999999999997, 'max_depth': 3, 'n_estimators': 79, 'subsample': 0.5, 'colsample_bytree': 0.7799999999999998}
# alco_params = {'base_score': 0.52, 'scale_pos_weight': 1.0, 'reg_lambda': 0.97, 'reg_alpha': 0.0, 'gamma': 0.19, 'max_depth': 3, 'n_estimators': 89, 'colsample_bytree': 1.0, 'subsample': 0.98}
# active_params = {'base_score': 0.52, 'scale_pos_weight': 0.98, 'reg_lambda': 0.99, 'reg_alpha': 0.019999999999999997, 'gamma': 0.16999999999999998, 'max_depth': 5, 'n_estimators': 82, 'colsample_bytree': 0.8899999999999999, 'subsample': 0.99}

In [373]:
smoke_estimator = search_xgb(X_helper, y_smoke, smoke_params)


[11s] -0.2390162587 {'reg_alpha': 0.01} | {'reg_alpha': [0.0, 0.01]}
[17s] -0.2390162587 {'colsample_bytree': 0.7799999999999998} | {'colsample_bytree': [0.7699999999999998, 0.7799999999999998, 0.7899999999999998]}
[11s] -0.2390162587 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.99, 1.0]}
[19s] -0.2388354244 {'max_depth': 3} | {'max_depth': [1, 2, 3]}
[54s] -0.2388354244 {'n_estimators': 79} | {'n_estimators': [76, 77, 78, 79, 80, 81, 82]}
[15s] -0.2388212301 {'reg_lambda': 0.99} | {'reg_lambda': [0.99, 1.0]}
[23s] -0.2387788451 {'base_score': 0.53} | {'base_score': [0.51, 0.52, 0.53]}
[16s] -0.2387788451 {'subsample': 0.5} | {'subsample': [0.5, 0.51]}
[25s] -0.2387788451 {'gamma': 0.02999999999999997} | {'gamma': [0.02999999999999997, 0.03999999999999997, 0.049999999999999975]}

[24s] -0.2387788451 {'colsample_bytree': 0.7799999999999998} | {'colsample_bytree': [0.7699999999999998, 0.7799999999999998, 0.7899999999999998]}
[54s] -0.2387788451 {'n_estimators': 79} | {'n_estimato

In [374]:
alco_estimator = search_xgb(X_helper, y_alco, alco_params)


[22s] -0.1923384277 {'reg_lambda': 0.99} | {'reg_lambda': [0.98, 0.99, 1.0]}
[14s] -0.1923384277 {'colsample_bytree': 1.0} | {'colsample_bytree': [0.99, 1.0]}
[54s] -0.1923239729 {'n_estimators': 90} | {'n_estimators': [85, 86, 87, 88, 89, 90, 91]}
[22s] -0.1923239729 {'max_depth': 3} | {'max_depth': [2, 3, 4]}
[16s] -0.1923239729 {'reg_alpha': 0.0} | {'reg_alpha': [0.0, 0.01]}
[22s] -0.1923229018 {'base_score': 0.53} | {'base_score': [0.53, 0.54, 0.55]}
[22s] -0.1923229018 {'subsample': 0.98} | {'subsample': [0.97, 0.98, 0.99]}
[15s] -0.1923229018 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.99, 1.0]}
[22s] -0.1923228143 {'gamma': 0.16999999999999998} | {'gamma': [0.14999999999999997, 0.15999999999999998, 0.16999999999999998]}

[15s] -0.1923228143 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.99, 1.0]}
[22s] -0.1923228143 {'subsample': 0.98} | {'subsample': [0.97, 0.98, 0.99]}
[23s] -0.1923228143 {'max_depth': 3} | {'max_depth': [2, 3, 4]}
[15s] -0.1923228143 {'reg_alpha

In [375]:
active_estimator = search_xgb(X_helper, y_active, active_params)


[32s] -0.4923239737 {'base_score': 0.52} | {'base_score': [0.5, 0.51, 0.52]}
[31s] -0.4923239737 {'reg_alpha': 0.019999999999999997} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[30s] -0.4923239737 {'subsample': 0.99} | {'subsample': [0.98, 0.99, 1.0]}
[31s] -0.4923239737 {'scale_pos_weight': 0.98} | {'scale_pos_weight': [0.97, 0.98, 0.99]}
[31s] -0.4923239737 {'gamma': 0.16999999999999998} | {'gamma': [0.15999999999999998, 0.16999999999999998, 0.18]}
[72s] -0.4922901218 {'n_estimators': 84} | {'n_estimators': [83, 84, 85, 86, 87, 88, 89]}
[30s] -0.4922901218 {'max_depth': 5} | {'max_depth': [4, 5, 6]}
[30s] -0.4922901218 {'colsample_bytree': 0.8899999999999999} | {'colsample_bytree': [0.8799999999999999, 0.8899999999999999, 0.8999999999999999]}
[30s] -0.4922901218 {'reg_lambda': 0.99} | {'reg_lambda': [0.98, 0.99, 1.0]}

[70s] -0.4922820367 {'n_estimators': 82} | {'n_estimators': [81, 82, 83, 84, 85, 86, 87]}
[29s] -0.4922820367 {'subsample': 0.99} | {'subsampl

### Predict `cardio`

In [377]:
# cardio_params = {'base_score': 0.51, 'gamma': 0.18, 'max_depth': 5, 'n_estimators': 80, 'reg_alpha': 0.019999999999999997, 'reg_lambda': 0.99, 'scale_pos_weight': 1.0, 'subsample': 1.0, 'colsample_bytree': 1.0}

In [378]:
cardio_estimator = search_xgb(X, y, cardio_params)
print_importances(cardio_estimator)


[34s] -0.5384644044 {'gamma': 0.18} | {'gamma': [0.16999999999999998, 0.18, 0.19]}
[23s] -0.5384644044 {'subsample': 1.0} | {'subsample': [0.99, 1.0]}
[23s] -0.5384644044 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.99, 1.0]}
[34s] -0.5384644044 {'max_depth': 5} | {'max_depth': [4, 5, 6]}
[33s] -0.5384644044 {'reg_alpha': 0.019999999999999997} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[33s] -0.5384644044 {'reg_lambda': 0.99} | {'reg_lambda': [0.98, 0.99, 1.0]}
[22s] -0.5384644044 {'colsample_bytree': 1.0} | {'colsample_bytree': [0.99, 1.0]}
[77s] -0.5384644044 {'n_estimators': 80} | {'n_estimators': [77, 78, 79, 80, 81, 82, 83]}
[33s] -0.5384644044 {'base_score': 0.51} | {'base_score': [0.5, 0.51, 0.52]}

[77s] -0.5384644044 {'n_estimators': 80} | {'n_estimators': [77, 78, 79, 80, 81, 82, 83]}
[22s] -0.5384644044 {'colsample_bytree': 1.0} | {'colsample_bytree': [0.99, 1.0]}
[34s] -0.5384644044 {'base_score': 0.51} | {'base_score': [0.5, 0.51, 0.52]}
[3

In [379]:
test = read_csv('test.csv')
X_helper_test = test.drop(['smoke', 'alco', 'active'], axis=1).values

# TODO: try with normal predict.
test['smoke'].fillna(pandas.Series(smoke_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)
test['alco'].fillna(pandas.Series(alco_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)
test['active'].fillna(pandas.Series(active_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)

numpy.savetxt(f'xgboost.txt', cardio_estimator.predict_proba(test.values)[:, 1], fmt='%f')